### DB 연결

In [1]:
import psycopg2

conn = psycopg2.connect(
  'host=localhost port=5432 dbname=postgres user=postgres password=postgres',
  options='-c search_path=sbd'
)
conn.set_session(autocommit=True)
cur = conn.cursor()

### 테이블 생성

In [2]:
cur.execute(
  f'''
  select count(*)
  from information_schema.tables
  where
    table_schema = 'sbd' and
    table_name ~ 'building_floor'
  '''
)
if not cur.fetchone()[0]:
  cur.execute(
    open('sql/sbd-create_table_building_floor.sql', 'r').read()
  )

후속 작업에 필요한 컬럼 목록, 테이블 초기화

In [3]:
cur.execute(
  f'''
  select column_name
  from information_schema.columns
  where 
    table_schema = 'sbd' and
    table_name = 'building_floor' and
    column_default is null
  '''
)
column_list = [
  col[0]
  for col
  in cur.fetchall()
]

### 업로드 함수

In [4]:
def uploadToDB(data_table):
  print(datetime.now(),'start...',end='')
  data_table.columns = [
    '관리_건축물대장_PK', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드', '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '새주소_도로_코드', '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '동_명', '층_구분_코드', '층_구분_코드_명', '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명', '기타_용도', '면적(㎡)', '주_부속_구분_코드', '주_부속_구분_코드_명', '면적_제외_여부', '생성_일자'
  ]
  data_table['시도_코드'] = [code[0:2] for code in data_table['시군구_코드']]
  data_table = data_table[data_table['시도_코드'] == '11'] # '전국' 중 '서울'만 추출
  data_table['pnu'] = data_table['시군구_코드'] + data_table['법정동_코드'] + [ # 토지 ID인 PNU 생성
    '1' if code == '0' else '2' if code == '1' else '0'
    for code
    in data_table['대지_구분_코드']
  ] + data_table['번'] + data_table['지']
  data_table['road_address'] = data_table['도로명_대지_위치'].str.strip()
  data_table = data_table[[
    'pnu', '관리_건축물대장_PK', '대지_위치', 'road_address', '건물_명', '동_명', '층_구분_코드', '층_구분_코드_명', '층_번호', '층_번호_명', '구조_코드', '구조_코드_명', '기타_구조', '주_용도_코드', '주_용도_코드_명', '기타_용도', '면적(㎡)', '주_부속_구분_코드_명', '면적_제외_여부', '생성_일자'
  ]]
  data_table.columns = column_list
  data_table.floor_no_name = data_table.floor_no_name.replace('[\\\]', '', regex=True) # 전처리 : 특수문자 제거
  data_table.structure_detail = data_table.structure_detail.replace('[\\\]', '', regex=True) # 전처리 : 특수문자 제거
  data_table.use_detail = data_table.use_detail.replace('[\\\]', '', regex=True) # 전처리 : 특수문자 제거
  data_table.is_excepted_area = [ # 0,1 -> False, True
    False if (pd.isna(tf) or tf == '0') else True
    for tf
    in data_table['is_excepted_area']
  ]
  data_table.to_csv( # .txt 파일로 로컬 저장
    'temp_building_floor.txt',
    sep='|',
    index=False,
    header=False,
    encoding='CP949'
  )
  temp_file = open('temp_building_floor.txt', 'r')
  print('upload...',end='')
  cur.copy_from( # file bulk insert
    temp_file,
    'building_floor',
    sep='|',
    columns=column_list,
    null=''
  )
  temp_file.close()
  os.remove('temp_building_floor.txt')
  print('end',datetime.now())

### 소스 데이터 업로드

In [5]:
from zipfile import ZipFile
import pandas as pd
import os
from datetime import datetime

zf = ZipFile('D:/data/building_register/building_floor/국토교통부_건축물대장_층별개요+(2022년+07월).zip')
source_chunks = pd.read_csv(
  zf.open('mart_djy_04.txt'),
  sep='|',
  encoding='CP949',
  header=None,
  dtype='string',
  chunksize=500_000
)

In [6]:
cur.execute(
  'delete from building_floor'
)

In [7]:
for source_dt in source_chunks:
  uploadToDB(source_dt)

2022-09-29 21:55:21.381551 start...

C:\Users\Bob\AppData\Local\Temp/ipykernel_27504/3267446478.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_table['pnu'] = data_table['시군구_코드'] + data_table['법정동_코드'] + [ # 토지 ID인 PNU 생성
C:\Users\Bob\AppData\Local\Temp/ipykernel_27504/3267446478.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_table['road_address'] = data_table['도로명_대지_위치'].str.strip()


upload...end 2022-09-29 21:55:24.586631
2022-09-29 21:55:29.583923 start...upload...end 2022-09-29 21:55:32.901242
2022-09-29 21:55:37.805385 start...upload...end 2022-09-29 21:55:41.233155
2022-09-29 21:55:46.037851 start...upload...end 2022-09-29 21:55:49.363081
2022-09-29 21:55:54.264306 start...upload...end 2022-09-29 21:55:57.909796
2022-09-29 21:56:02.619094 start...upload...end 2022-09-29 21:56:05.976123
2022-09-29 21:56:10.733457 start...upload...end 2022-09-29 21:56:14.047389
2022-09-29 21:56:18.876682 start...upload...end 2022-09-29 21:56:22.723444
2022-09-29 21:56:27.549739 start...upload...end 2022-09-29 21:56:31.173844
2022-09-29 21:56:35.983336 start...upload...end 2022-09-29 21:56:39.274022
2022-09-29 21:56:44.154397 start...upload...end 2022-09-29 21:56:47.566048
2022-09-29 21:56:52.336833 start...upload...end 2022-09-29 21:56:55.595159
2022-09-29 21:57:00.458510 start...upload...end 2022-09-29 21:57:03.868937
2022-09-29 21:57:08.719176 start...upload...end 2022-09-29 2